# AWS Lambda를 MCP화하기
## Bedrock AgentCore Gateway를 사용하여 AWS Lambda 함수를 안전한 MCP 도구로 변환

## 개요
Bedrock AgentCore Gateway는 고객이 인프라나 호스팅을 관리할 필요 없이 기존 AWS Lambda 함수를 완전 관리형 MCP 서버로 전환할 수 있는 방법을 제공합니다. Gateway는 이러한 모든 도구에 걸쳐 통일된 Model Context Protocol(MCP) 인터페이스를 제공합니다. Gateway는 들어오는 요청과 대상 리소스에 대한 아웃바운드 연결 모두에 대해 안전한 액세스 제어를 보장하기 위해 이중 인증 모델을 사용합니다. 프레임워크는 두 가지 주요 구성 요소로 구성됩니다: 게이트웨이 대상에 액세스하려는 사용자를 검증하고 권한을 부여하는 Inbound Auth와 인증된 사용자를 대신하여 게이트웨이가 백엔드 리소스에 안전하게 연결할 수 있도록 하는 Outbound Auth입니다. Gateway는 아웃바운드 권한 부여를 위해 IAM 역할을 사용하여 AWS Lambda 함수에 대한 호출을 권한 부여합니다.

![작동 방식](images/lambda-iam-gateway.png)

### 튜토리얼 세부 정보


| 정보                 | 세부 사항                                                 |
|:---------------------|:----------------------------------------------------------|
| 튜토리얼 유형           | 대화형                                                    |
| AgentCore 구성 요소    | AgentCore Gateway, AgentCore Identity                     |
| 에이전트 프레임워크      | Strands Agents                                            |
| Gateway 대상 유형      | AWS Lambda                                                |
| 인바운드 인증 IdP      | Amazon Cognito                                            |
| 아웃바운드 인증        | AWS IAM                                                   |
| LLM 모델             | Anthropic Claude Sonnet 3.7, Amazon Nova Pro              |
| 튜토리얼 구성 요소      | AgentCore Gateway 생성 및 AgentCore Gateway 호출            |
| 튜토리얼 분야          | 교차 분야                                                 |
| 예제 복잡성           | 쉬움                                                      |
| 사용된 SDK           | boto3                                                     |

튜토리얼의 첫 번째 부분에서는 몇 가지 AmazonCore Gateway 대상을 생성할 것입니다

### 튜토리얼 아키텍처
이 튜토리얼에서는 AWS Lambda 함수에 정의된 작업을 MCP 도구로 변환하고 Bedrock AgentCore Gateway에서 호스팅할 것입니다.
데모 목적으로 Amazon Bedrock 모델을 사용하는 Strands Agent를 사용할 것입니다
예제에서는 get_order와 update_order라는 두 가지 도구를 가진 매우 간단한 에이전트를 사용할 것입니다.

## 전제 조건

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Jupyter notebook (Python 커널)
* uv
* AWS 자격 증명
* Amazon Cognito

## 들어오는 AgentCore Gateway 요청에 대한 인증 구성
AgentCore Gateway는 인바운드 및 아웃바운드 인증을 통해 안전한 연결을 제공합니다. 인바운드 인증의 경우 AgentCore Gateway는 호출 중에 전달된 OAuth 토큰을 분석하여 게이트웨이의 도구에 대한 액세스를 허용하거나 거부할지 결정합니다. 도구가 외부 리소스에 액세스해야 하는 경우 AgentCore Gateway는 API 키, IAM 또는 OAuth 토큰을 통한 아웃바운드 인증을 사용하여 외부 리소스에 대한 액세스를 허용하거나 거부할 수 있습니다.



During the inbound authorization flow, an agent or the MCP client calls an MCP tool in the AgentCore Gateway adding an OAuth access token (generated from the user’s IdP). AgentCore Gateway then validates the OAuth access token and performs inbound authorization.

AgentCore Gateway에서 실행되는 도구가 외부 리소스에 액세스해야 하는 경우 OAuth는 Gateway 대상에 대한 리소스 자격 증명 공급자를 사용하여 다운스트림 리소스의 자격 증명을 검색합니다. AgentCore Gateway는 권한 부여 자격 증명을 호출자에게 전달하여 다운스트림 API에 액세스할 수 있도록 합니다. 

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Amazon SageMaker 노트북을 사용하지 않는 경우 AWS 자격 증명 설정import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # 액세스 키 설정
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # 시크릿 키 설정
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1') # AWS 리전 설정

In [ ]:
import os
import sys

# 현재 스크립트의 디렉토리 가져오기if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__이 정의되지 않은 경우의 대안 (예: Jupyter)

# utils.py가 포함된 디렉토리로 이동 (한 단계 위)utils_dir = os.path.abspath(os.path.join(current_dir, '..'))

# sys.path에 추가sys.path.insert(0, utils_dir)

# 이제 utils를 가져올 수 있습니다import utils

In [ ]:
#### MCP 도구로 변환하려는 샘플 AWS Lambda 함수 생성lambda_resp = utils.create_gateway_lambda("lambda_function_code.zip")

if lambda_resp is not None:    if lambda_resp['exit_code'] == 0:
        print("Lambda 함수가 다음 ARN으로 생성되었습니다: ", lambda_resp['lambda_function_arn'])
    else:
        print("Lambda 함수 생성이 다음 메시지와 함께 실패했습니다: ", lambda_resp['lambda_function_arn'])

In [ ]:
#### Gateway가 맡을 IAM 역할 생성import utils
agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore 게이트웨이 역할 ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway에 대한 인바운드 권한 부여를 위한 Amazon Cognito 풀 생성

In [ ]:
# Cognito 사용자 풀 생성import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Cognito 리소스 생성 또는 검색 중...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"사용자 풀 ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("리소스 서버가 확인되었습니다.")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"클라이언트 ID: {client_id}")

# 검색 URL 가져오기cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# 인바운드 권한 부여를 위한 Amazon Cognito Authorizer로 Gateway 생성

In [ ]:
# CMK 없이 Cognito 권한 부여자로 Gateway 생성. 이전 단계에서 생성한 Cognito 사용자 풀 사용gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # 클라이언트는 Cognito에서 구성된 ClientId와 일치해야 합니다. 예: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='TestGWforLambda',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAM 역할은 Gateway를 생성/나열/가져오기/삭제할 권한이 있어야 합니다 
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with AWS Lambda target type'
)
print(create_response)
# GatewayTarget 생성에 사용되는 GatewayID 검색gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# AWS Lambda 대상 생성 및 MCP 도구로 변환

In [ ]:
# 아래의 AWS Lambda 함수 ARN을 교체하세요lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_resp['lambda_function_arn'], # 이것을 귀하의 AWS Lambda 함수 ARN으로 교체하세요
            "toolSchema": {
                "inlinePayload": [
                    {
                        "name": "get_order_tool",
                        "description": "주문을 가져오는 도구",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    },                    
                    {
                        "name": "update_order_tool",
                        "description": "주문 ID를 업데이트하는 도구",
                        "inputSchema": {
                            "type": "object",
                            "properties": {
                                "orderId": {
                                    "type": "string"
                                }
                            },
                            "required": ["orderId"]
                        }
                    }
                ]
            }
        }
    }
}

credential_config = [ 
    {
        "credentialProviderType" : "GATEWAY_IAM_ROLE"
    }
]
targetname='LambdaUsingSDK'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='Lambda Target using SDK',
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config)

# Strands Agent에서 Bedrock AgentCore Gateway 호출

Strands 에이전트는 Model Context Protocol(MCP) 사양을 구현하는 Bedrock AgentCore Gateway를 통해 AWS 도구와 원활하게 통합됩니다. 이 통합은 AI 에이전트와 AWS 서비스 간의 안전하고 표준화된 통신을 가능하게 합니다.

핵심적으로 Bedrock AgentCore Gateway는 기본적인 MCP API인 ListTools와 InvokeTools를 노출하는 프로토콜 준수 Gateway 역할을 합니다. 이러한 API를 통해 MCP 호환 클라이언트나 SDK는 안전하고 표준화된 방식으로 사용 가능한 도구를 발견하고 상호 작용할 수 있습니다. Strands 에이전트가 AWS 서비스에 액세스해야 할 때 이러한 MCP 표준화된 엔드포인트를 사용하여 Gateway와 통신합니다.

Gateway의 구현은 (MCP 권한 부여 사양)[https://modelcontextprotocol.org/specification/draft/basic/authorization]을 엄격히 준수하여 강력한 보안과 액세스 제어를 보장합니다. 이는 Strands 에이전트의 모든 도구 호출이 권한 부여 단계를 거치므로 강력한 기능을 활성화하면서 보안을 유지한다는 것을 의미합니다.

예를 들어, Strands 에이전트가 MCP 도구에 액세스해야 할 때 먼저 ListTools를 호출하여 사용 가능한 도구를 발견한 다음 InvokeTools를 사용하여 특정 작업을 실행합니다. Gateway는 필요한 모든 보안 검증, 프로토콜 변환 및 서비스 상호 작용을 처리하여 전체 프로세스를 원활하고 안전하게 만듭니다.

이러한 아키텍처 접근 방식은 MCP 사양을 구현하는 모든 클라이언트나 SDK가 Gateway를 통해 AWS 서비스와 상호 작용할 수 있음을 의미하며, AI 에이전트 통합을 위한 다재다능하고 미래 지향적인 솔루션이 됩니다.

![Strands agent calling Gateway](images/strands-lambda-gateway.png)

# 인바운드 권한 부여를 위한 Amazon Cognito에서 액세스 토큰 요청

In [ ]:
import time
time.sleep(10)

In [ ]:
print("Amazon Cognito 권한 부여자에서 액세스 토큰 요청 중... 도메인 이름 전파가 완료될 때까지 일정 시간 동안 실패할 수 있습니다")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("토큰 응답:", token)

# Bedrock AgentCore Gateway를 사용하여 AWS Lambda의 MCP 도구를 호출하는 Strands 에이전트

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## ~/.aws/credentials에 구성된 IAM 자격 증명은 Bedrock 모델에 액세스할 수 있어야 합니다yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# 루트 strands 로거를 구성합니다. 문제를 디버깅하는 경우 DEBUG로 변경하세요.logging.getLogger("strands").setLevel(logging.INFO)

# 로그를 보기 위한 핸들러 추가logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
# listTools 호출    tools = client.list_tools_sync()
# 모델과 도구로 에이전트 생성    agent = Agent(model=yourmodel,tools=tools) ## 원하는 모델로 교체할 수 있습니다
    print(f"에이전트에 로드된 도구는 {agent.tool_names}")
    # print(f"Tools configuration in the agent are {agent.tool_config}")
    # 샘플 프롬프트로 에이전트를 호출합니다. 이는 MCP listTools만 호출하고 LLM이 액세스할 수 있는 도구 목록을 검색합니다. 아래는 실제로 어떤 도구도 호출하지 않습니다.
    agent("안녕하세요, 사용 가능한 모든 도구를 나열해 주실 수 있나요?")
# 샘플 프롬프트로 에이전트를 호출하고, 도구를 호출하여 응답을 표시합니다    agent("주문 ID 123의 주문 상태를 확인하고 도구의 정확한 응답을 보여주세요")
# MCP 도구를 명시적으로 호출합니다. MCP 도구 이름과 인수는 AWS Lambda 함수 또는 OpenAPI/Smithy API와 일치해야 합니다    result = client.call_tool_sync(
    tool_use_id="get-order-id-123-call-1", # 이것을 고유 식별자로 교체할 수 있습니다. 
    name=targetname+"___get_order_tool", # 이는 AWS Lambda 대상 유형을 기반으로 한 도구 이름입니다. 대상 이름에 따라 변경됩니다
    arguments={"orderId": "123"}
    )
# MCP 도구 응답 출력    print(f"도구 호출 결과: {result['content'][0]['text']}")


**문제: 아래 셀을 실행할 때 아래 오류가 발생하면 pydantic과 pydantic-core 버전 간의 비호환성을 나타냅니다.**

```
TypeError: model_schema()가 예상치 못한 키워드 인수 'generic_origin'을 받았습니다
```
**해결 방법은?**

호환되는 pydantic==2.7.2와 pydantic-core 2.27.2를 모두 설치해야 합니다. 완료되면 커널을 다시 시작하세요.

# 정리

IAM 역할, IAM 정책, 자격 증명 공급자, AWS Lambda 함수, Cognito 사용자 풀, S3 버킷과 같은 추가 리소스도 생성되며, 정리 과정에서 수동으로 삭제해야 할 수 있습니다. 이는 실행하는 예제에 따라 다릅니다.

## 게이트웨이 삭제 (선택사항)

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)